In [2]:
# calc_iters.py
import math, torch, argparse, pandas as pd



N = 2000
P = 3
S = 16 * 1 * P
iters = math.ceil(2 * N / (6 * S))

print(f"Dataset={N}, GPUs={P}, effective_batch={S}")
print(f"→ Set --num_iterations {iters}")


Dataset=2000, GPUs=3, effective_batch=48
→ Set --num_iterations 14


In [ ]:
# calc_iters.py
import math, torch, argparse, pandas as pd

ap = argparse.ArgumentParser()
ap.add_argument("--csv", required=True)
ap.add_argument("--epochs", type=float, default=2)
ap.add_argument("--num_generations", type=int, default=4)
ap.add_argument("--batch", type=int, default=2)
ap.add_argument("--accum", type=int, default=4)
args = ap.parse_args()

N = len(pd.read_csv(args.csv))
P = torch.cuda.device_count() or 1
S = args.batch * args.accum * P
iters = math.ceil(args.epochs * N / (args.num_generations * S))

print(f"Dataset={N}, GPUs={P}, effective_batch={S}")
print(f"→ Set --num_iterations {iters}")


In [1]:
import os
import json
import random
import torch
import argparse
import numpy as np
import pandas as pd
import wandb
import logging
from pathlib import Path
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    set_seed,
    TrainerCallback,
    EarlyStoppingCallback
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType
)

set_seed(42)

/home/jesseliu/miniconda3/envs/maze/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"
user_token = os.getenv("HF_API_TOKEN")


tokenizer = AutoTokenizer.from_pretrained(model_name, token=user_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print(tokenizer.chat_template)

{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- if strftime_now is defined %}
        {%- set date_string = strftime_now("%d %b %Y") %}
    {%- else %}
        {%- set date_string = "26 Jul 2024" %}
    {%- endif %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{{- "Cutting Knowledge Date: December 2023\n" }}
{{- 